In [ ]:
%pip install pymogwai

  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached pydot-3.0.2-py3-none-any.whl.metadata (10 kB)
  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached pysemanticslides-0.0.10-py3-none-any.whl.metadata (3.0 kB)
  Using cached bibtexparser-1.4.2-py3-none-any.whl
  Using cached dacite-1.8.1-py3-none-any.whl.metadata (15 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached isbnlib-3.10.14-py2.py3-none-any.whl.metadata (16 kB)
  Using cached pygenericspreadsheet-0.4.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached pylodstorage-0.13.3-py3-none-any.whl.metadata (3.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached dicttoxml2-2.1.0-py3-none-any

see [pymogwai issue #2](https://github.com/juupje/pyMogwai/issues/2) for how to setup the pymogwai environment

In [4]:
from mogwai.core.traversal import MogwaiGraphTraversalSource
from mogwai.core.steps.statics import *
from mogwai.parser import graphml_to_mogwaigraph

Then, load the graph!

In [5]:
airroutes_path = "air-routes-latest.graphml"
graph = graphml_to_mogwaigraph(airroutes_path, node_label_key="labelV", 
            edge_label_key="labelE", node_name_key="desc")

Finally, create a traversal source

In [6]:
g = MogwaiGraphTraversalSource(graph)

## Let's go on a holiday!
We want to go on holiday, where can we go from germany?

In [7]:
query = g.V().has_label("country").has_name("Germany")\
        .out("contains")\
        .out("route")\
        .in_("contains")\
        .has_label("country").name().dedup().to_list()
print("Query:",query.print_query())
res = query.run()
print("Result:", res[:10], "total: ", len(res))

Query: V -> Contains -> Has -> Out -> Out -> In -> Contains -> Values -> Dedup -> ToList
Result: ['United States', 'Puerto Rico', 'Canada', 'United Kingdom', 'France', 'Finland', 'Japan', 'Singapore', 'United Arab Emirates', 'India'] total:  110


Okay, that's a bit much. Let's see what's the farthest we can get without a layover

In [8]:
query = g.V().has_label("country").has_name("Germany")\
        .out("contains")\
        .outE("route").as_("e")\
        .inV()\
        .in_("contains").has_label("country").as_('dest')\
        .select('e').properties('dist').max_()\
        .select('dest').by('name').next()
print("Result:", query.run())

Result: Argentina


Well, that's nice. Let's go there! But from which airport are we departing?

In [9]:
query = g.V().has_label("country").has_name("Germany")\
            .out("contains").as_('start')\
            .outE("route").as_("e")\
            .inV()\
            .in_("contains").has_label("country").as_('dest')\
            .select('e').properties('dist').as_('dist').max_().\
            select('start', 'dest', 'dist').to_list().by('name')
res = query.run()
print("Result:",res)

Result: [['Frankfurt am Main', 'Argentina', 7141]]


Hmm, that's a bit far away from Aachen. Let's see where we can go from a nearby airport

In [10]:
g.V().has_label("airport").within("city", ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start')\
            .outE("route").as_("e")\
            .inV()\
            .in_("contains").has_label("country").as_('dest')\
            .order(desc=True).by(select('e').properties('dist'))\
            .limit(5)\
            .select('start', 'dest').to_list().by('name').run()

[['Dusseldorf International Airport', 'Singapore'],
 ['Brussels Airport', 'Japan'],
 ['Brussels South Charleroi Airport', 'Hong Kong'],
 ['Brussels Airport', 'Hong Kong'],
 ['Brussels Airport', 'China']]

Those flights are really long...
I guess we have to accept layovers then. Where can we go within three hops if we want to depart from Maastricht Aachen Airport? We don't want a flight longer than 2000 miles.

In [11]:
g.V().has_label('airport').has_property("city", "Maastricht").as_('start')\
    .repeat(
        outE('route').as_('e')\
        .inV().filter_(
            select('e').properties('dist').is_(lte(2000))
        ).simple_path()
    ).times(3).emit().as_('dest')\
    .path().as_('p')\
    .count(Scope.local).order(asc=True).as_('length')\
    .select('dest').dedup()\
    .select('dest', 'length').to_list().by('city').run()

AttributeError: 'Traversal' object has no attribute 'has_property'